In [1]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
from keras.layers import Input, Dense, convolutional,core,Flatten,Reshape,Concatenate
from keras.models import Model,load_model
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.layers.convolutional import Convolution2D

from scipy.signal import convolve2d



standardise = lambda rewards: (rewards - np.mean(rewards,keepdims =1)) / np.std(rewards ) ;


class temp:
    def __init__(self):
        self.clean();
    def clean(self):
        self.states = []
        self.gradients = []
        self.rewards = []
        self.probs = []
        self.tpreds = [];
        self.aprobs = [];
        self.actions = [];
    def remember(self,state,action,prob,reward,tpred):
        self.states.append(state);
        self.probs.append(prob)
        self.rewards.append(reward);
        self.tpreds.append(tpred);
        y = np.zeros([action_size])
        y[action] = 1
        self.actions.append(y);
        self.gradients.append(np.array(y).astype('float32') - prob)
        
class PGAgent:
    def __init__(self, state_size, action_size, tmp):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
#         self.states = []
#         self.gradients = []
#         self.rewards = []
#         self.probs = []
        self.tmp = tmp;
        self.model = self._build_model()
        self.summary = self.model.summary;

    def _build_model(self):
        model = Sequential()
        model.add(Reshape((80, 80, 1), input_shape=(self.state_size,)))
        model.add(Convolution2D(25, (6, 6), subsample=(3, 3), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Convolution2D(5, (6, 6), subsample=(1, 1), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Flatten())
        model.add(Dense(20, activation='relu', init='he_uniform'))
        model.add(Dense(20, activation='relu', init='he_uniform'))
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

    def remember(self, state, action, prob, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.gradients.append(np.array(y).astype('float32') - prob)
        self.states.append(state)
        self.rewards.append(reward)

    def act(self, state):
        # state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        self.tmp.aprobs.append(aprob)
        prob = aprob / np.sum(aprob)
        action = np.random.choice(self.action_size, 1, p=prob)[0]
        return action, prob

    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0:
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    def train(self,rewards):
        gradients = np.vstack(self.tmp.gradients)
        # rewards = np.vstack(self.rewards)
        # rewards = self.discount_rewards(rewards)
#         rewards = (rewards - np.mean(rewards,keepdims=1)) / np.std(rewards)
        gradients *= rewards
        X = np.squeeze(np.vstack([self.tmp.states]))
        Y = self.tmp.aprobs + self.learning_rate * np.squeeze(np.vstack([gradients]))
        self.model.train_on_batch(X, Y)

    def clean(self): 
        self.tmp.clean();
#         self.states, self.probs, self.gradients, self.rewards = [], [], [], []

    def load(self, name):
        self.model.load_weights(name)
        

    def save(self, name):
        self.model.save_weights(name)
        
    def readlog(self, LogName):
#         LogName = self.LogName;
        with open(LogName,'rb') as f:
                first = f.readline()      # Read the first line.
                f.seek(-2, 2)             # Jump to the second last byte.
                while f.read(1) != b"\n": # Until EOL is found...
                    f.seek(-2, 1)         # ...jump back the read byte plus one more.
                last = f.readline() 
                lst = last.split('\t');
                eind = lst.index('Episode')+1;
                self.episode = int(lst[eind]);
        
def preprocess(I):
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float).ravel()
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size,temp())
    
agent.summary();

Using TensorFlow backend.
[2017-06-12 17:21:27,247] Making new env: Pong-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 25)        925       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 5)         4505      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3645)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                72920     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 126       
Total para

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


In [2]:
class critic:
    def __init__(self,state_size,action_size,tmp):
        self.D = state_size;
        self.action_size = action_size;
        self.H = 5;
        self.tmp = tmp;
        self.model = self.model_init();
        self.predict = self.model.predict;
        self.summary = self.model.summary;
        self.load = self.model.load_weights;
        self.save = self.model.save_weights;
        
    def model_init(self):
        def lossfunc(y_true,y_pred):
            return K.mean(K.mean( K.square( y_pred / (K.abs(y_true)+1) - 1)  )); 

        D = self.D;
        H = self.H;
        x_input = Input(shape=(D,));
        po_input = Input(shape=(self.action_size,));
        conv1 = convolutional.Conv2D(filters=H,
                             kernel_size=(6,6),
                            strides=(3,3),
                            padding='same',
                            activation='relu')(Reshape((80, 80,1))(x_input))
        den1 = Flatten()(
            Dense(units=5*H,
             activation='relu')(conv1)
            )
        den1c = Concatenate()([den1,po_input]);
        
        score = Dense(units=2,
              activation = 'relu')(den1c)
    
        model = Model(inputs=[x_input,po_input], outputs=[score])
#         optimiser = keras.optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer='rmsprop',
              loss=lossfunc)
        return model
    
    def time_rewards(self,r):
        """ take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        grad = 0;
        for t in reversed(xrange(0, r.size)):
    #         grad = grad * gamma + r[t] ;
            if r[t] != 0: 
                running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
                grad = 2 * (r[t] > 0) - 1; 
            running_add = running_add + grad;
            discounted_r[t] = running_add
        return discounted_r
    
    def decouple(self,tpr):
        tpr1=np.maximum(tpr,0);
        tpr2=np.minimum(tpr,0);
        tpr1[tpr1==0]=np.maximum(np.max(tpr1),20);
        tpr2[tpr2==0]=np.minimum(np.min(tpr2),-20);
        tpr2=-tpr2;
        time_epr = np.hstack([tpr1,tpr2])
        return(time_epr)
    
    def get_rsignal(self):
        x = np.vstack(self.tmp.tpreds);
        x_diff = convolve2d(x,[[0.],[-1.],[1.]],mode='same');
        x_mean = convolve2d(x,[[0.],[.5],[.5]],mode='same');
        x_vari = abs(x_diff)/x_mean;
        x_vari[np.isnan(x_vari)]=0;
        x_vari = np.mean(x_vari,axis = 1,keepdims = 1);
        return(x_vari);
    #     sx_avg = x_avg * rsign;
    def train(self):
        epx = np.vstack(self.tmp.states);
        epy = np.vstack(self.tmp.actions);
        epr = np.vstack(self.tmp.rewards);
        tpr=self.time_rewards(epr);
        time_epr = self.decouple(tpr);
        curr_loss = self.model.train_on_batch([epx,epy], time_epr);
        return curr_loss
        
#     def save(self,name):
#         print('no method to save');
#     def load(self,name):
#         print('no method to load')
cagent = critic(state_size,action_size,temp());
cagent.model_init();
cagent.summary();
pbatch = lambda x: np.reshape(x,(1,-1))
# tpred = cagent.predict([pbatch(x),pbatch(prob)])
# print(tpred)
# tpred = cagent.predict([x,prob]);


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 6400)          0                                            
____________________________________________________________________________________________________
reshape_2 (Reshape)              (None, 80, 80, 1)     0           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 27, 27, 5)     185         reshape_2[0][0]                  
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 27, 27, 25)    150         conv2d_3[0][0]                   
___________________________________________________________________________________________

In [ ]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size,temp())
    
agent.summary();
state_size,action_size

[2017-06-12 17:21:28,211] Making new env: Pong-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 27, 27, 25)        925       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 5)         4505      
_________________________________________________________________
flatten_4 (Flatten)          (None, 3645)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 20)                72920     
_________________________________________________________________
dense_9 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 126       
Total para

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


(6400, 6)

In [ ]:
AgentName = 'ac1'
AgentFile = 'Models/%s.h5'%AgentName;
cAgentFile = 'Models/%s-critic.h5'%AgentName;

LogName = 'Models/%s.log'%AgentName;
# LogFile = open(LogName,'a');
resume = 0;
render = 0;

if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    tmp = temp();
    agent = PGAgent(state_size, action_size,tmp)
    cagent = critic(state_size,action_size,tmp);
#     agent.summary();
#     cagent.summary();
    

    if resume:
        agent.load(AgentFile)
        agent.readlog(LogName);
        episode = agent.episode;
        cagent.load(cAgentFile)
        
    while episode > -1:
        if render:
            env.render()

        cur_x = preprocess(state)
        x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
        prev_x = cur_x

        action, prob = agent.act(np.expand_dims(x,1).T)
        avct = np.zeros([action_size])
        avct[action] = 1

        state, reward, done, info = env.step(action)
        score += reward
        ### manage the list 
#         agent.remember(x, action, prob, reward)
        tpred = cagent.predict([pbatch(x),pbatch(avct)]);
        tmp.remember(x,action,prob,reward,tpred);
#         cagent.remember(x, action, prob, reward, tpred);
        
        if done:
            episode += 1
            rewards = np.vstack(tmp.rewards)
            
            
            ## process reward on a per event basis
            curr_loss = cagent.train();
#             losshist.add(episode_number,curr_loss);                        
#             rsignal = merge_signal(epr,eptpred,.5);
            rewards = standardise(agent.discount_rewards(rewards) * cagent.get_rsignal())
            
            agent.train(rewards)
            tmp.clean();
#             agent.record()
            msg = '%s\t%d\t%s\t%f\t%s\t%f' % ('Episode',episode,'Score', score,'Loss',curr_loss);
            print(msg);
            if episode == 1  and resume == 0:
                open(LogName,'w').close();
            with open(LogName,'a+') as LogFile:
                LogFile.write(msg+'\n');
            
            score = 0
            state = env.reset()
            prev_x = None
            if episode > 1 and episode % 10 == 0:
                agent.save(AgentFile)
                cagent.save(cAgentFile)

[2017-06-12 17:21:28,841] Making new env: Pong-v0
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.p

Episode	1	Score	-20.000000	Loss	0.999573
Episode	2	Score	-20.000000	Loss	0.991180
Episode	3	Score	-20.000000	Loss	0.978813
Episode	4	Score	-21.000000	Loss	0.907223
Episode	5	Score	-21.000000	Loss	0.840776
Episode	6	Score	-20.000000	Loss	0.846171
Episode	7	Score	-21.000000	Loss	0.688707
Episode	8	Score	-20.000000	Loss	0.748515
Episode	9	Score	-19.000000	Loss	0.770432
Episode	10	Score	-21.000000	Loss	0.586274
Episode	11	Score	-21.000000	Loss	0.515917
Episode	12	Score	-20.000000	Loss	0.778500
Episode	13	Score	-21.000000	Loss	0.500952
Episode	14	Score	-19.000000	Loss	0.687984
Episode	15	Score	-20.000000	Loss	0.780151
Episode	16	Score	-21.000000	Loss	0.443985
Episode	17	Score	-21.000000	Loss	0.392968
Episode	18	Score	-20.000000	Loss	0.664151
Episode	19	Score	-20.000000	Loss	0.644038
Episode	20	Score	-20.000000	Loss	0.610484
Episode	21	Score	-21.000000	Loss	0.351758
Episode	22	Score	-20.000000	Loss	0.686612
Episode	23	Score	-20.000000	Loss	0.607621
Episode	24	Score	-21.000000	Loss	0.330386
E

In [ ]:
print >> LogFile, 'Hi'

In [ ]:
LogFile = open('Models/%s.log'%AgentName,'a')
LogFile.write('Hi')